# <center> Предсказание победителя в Dota 2
<center> <img src="https://meduza.io/impro/YnJZAHUW6WHz_JQm1uRPkTql_qAhbfxt3oFJLGH7CJg/fill/980/0/ce/1/aHR0cHM6Ly9tZWR1/emEuaW8vaW1hZ2Uv/YXR0YWNobWVudHMv/aW1hZ2VzLzAwNy8x/NTcvNjk1L29yaWdp/bmFsL0tMVThLbUti/ZG5pSzlibDA0Wmlw/WXcuanBn.webp" width="700" height="700">

[Почитать подбробнее](https://meduza.io/feature/2021/10/19/rossiyskaya-komanda-vyigrala-chempionat-mira-po-dota-2-i-poluchila-18-millionov-dollarov-postoyte-otkuda-takie-dengi-neuzheli-igrat-v-dotu-tak-slozhno)

#### [Оригинальная статья](https://arxiv.org/pdf/2106.01782.pdf)
    

## **Краткий отчет**:

1) Подбирала фичи - убрала пару ненужных, сделала one-hot encoding категориальных, добавила одну новую фичу

2) Подбирала гиперпараметры для случайного леса, n_estimatores подбирала в конце

3) Пробовала другие модели: catboost, lightgbm

Catboost требовал меньше всего внимания и взаимодействия, но в итоге дал лучший результат на тесте. На кросс-валидации на трейне лучше всего был lightgbm после optuna, но на тесте отстал от catboost...

### Импорты

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [160]:
import os
import json
import pandas as pd
import datetime
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import (train_test_split, ShuffleSplit, 
                                     cross_val_score,  KFold, cross_validate)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from IPython.display import clear_output

%matplotlib inline

In [161]:
SEED = 10801
sns.set_style(style="whitegrid")
plt.rcParams["figure.figsize"] = 12, 8
warnings.filterwarnings("ignore")

## <left>Описание данных

Файлы:

- `sample_submission.csv`: пример файла-посылки
- `train_raw_data.jsonl`, `test_raw_data.jsonl`: "сырые" данные 
- `train_data.csv`, `test_data.csv`: признаки, созданные авторами
- `train_targets.csv`: результаты тренировочных игр

## <left>Описание признаков
    
Набор простых признаков, описывающих игроков и команды в целом

In [162]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [163]:
#PATH_TO_DATA = "../input/bi-ml-competition-2023"
PATH_TO_DATA = "/content/gdrive/MyDrive/dota"

df_train_features = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                             "train_data.csv"), 
                                index_col="match_id_hash")
df_train_targets = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                            "train_targets.csv"), 
                                   index_col="match_id_hash")

df_test_features = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                             "test_data.csv"), 
                                index_col="match_id_hash")

In [164]:
df_train_features.head()

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,15,7,2,0,7,...,0.000000,0,0,0,0,0.00,0,0,0,0
6db558535151ea18ca70a6892197db41,21,23,0,0,0,101,0,0,0,0,...,0.000000,0,0,0,0,0.00,0,0,0,0
19c39fe2af2b547e48708ca005c6ae74,160,22,7,0,0,57,0,0,0,1,...,0.000000,0,0,0,0,0.00,0,0,0,0
c96d629dc0c39f0c616d1949938a6ba6,1016,22,0,1,0,119,0,3,3,5,...,8.264696,0,0,3,0,0.25,0,0,3,0
156c88bff4e9c4668b0f53df3d870f1b,582,22,7,2,2,12,3,1,2,9,...,15.762911,3,1,0,1,0.50,0,0,3,0


Имеем ~32 тысячи наблюдений, каждое из которых характеризуется уникальным `match_id_hash` (захэшированное id матча), и 245 признаков. `game_time` показывает момент времени, в который получены эти данные. То есть по сути это не длительность самого матча, а например, его середина, таким образом, в итоге мы сможем получить модель, которая будет предсказывать вероятность победы каждой из команд в течение матча (хорошо подходит для букмекеров).

Нас интересует поле `radiant_win` (так называется одна из команд, вторая - dire). Остальные колоки здесь по сути получены из "будущего" и есть только для тренировочных данных, поэтому на них можно просто посмотреть).

In [ ]:
df_train_targets.head()

,game_time,radiant_win,duration,time_remaining,next_roshan_team
match_id_hash,,,,,
b9c57c450ce74a2af79c9ce96fac144d,658,True,1154,496,NaN
6db558535151ea18ca70a6892197db41,21,True,1503,1482,Radiant
19c39fe2af2b547e48708ca005c6ae74,160,False,2063,1903,NaN
c96d629dc0c39f0c616d1949938a6ba6,1016,True,2147,1131,Radiant
156c88bff4e9c4668b0f53df3d870f1b,582,False,1927,1345,Dire


## <left>Что есть в json файлах?

Описание сырых данных можно найти в `train_matches.jsonl` и `test_matches.jsonl`. Каждый файл содержит одну запись для каждого матча в [JSON](https://en.wikipedia.org/wiki/JSON) формате. Его легко превратить в питоновский объект при помощи метода `json.loads`.

In [ ]:
with open(os.path.join(PATH_TO_DATA, "train_raw_data.jsonl")) as fin:
    # прочтем 419 строку
    for i in range(419):
        line = fin.readline()
    
    # переведем JSON в питоновский словарь 
    match = json.loads(line)

In [ ]:
player = match["players"][9]
player["kills"], player["deaths"], player["assists"]

(0, 5, 5)

KDA - может быть неплохим признаком, этот показатель считается как:
    
<center>$KDA = \frac{K + A}{D}$

#### Сделаем чтение файла с сырыми данными и добавление новых признаков удобным

В этот раз для чтение `json` файлов лучше использовать библиотеку `ujson`, иначе все будет слишком долго :(

In [ ]:
!pip install ujson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 5.8 MB/s eta 0:00:00


In [165]:
try:
    import ujson as json
except ModuleNotFoundError:
    import json
    print ("Подумайте об установке ujson, чтобы работать с JSON объектами быстрее")
    
try:
    from tqdm.notebook import tqdm
except ModuleNotFoundError:
    tqdm_notebook = lambda x: x
    print ("Подумайте об установке tqdm, чтобы следить за прогрессом")

    
def read_matches(matches_file, total_matches=31698, n_matches_to_read=None):
    """
    Аргуент
    -------
    matches_file: JSON файл с сырыми данными
    
    Результат
    ---------
    Возвращает записи о каждом матче
    """
    
    if n_matches_to_read is None:
        n_matches_to_read = total_matches
        
    c = 0
    with open(matches_file) as fin:
        for line in tqdm(fin, total=total_matches):
            if c >= n_matches_to_read:
                break
            else:
                c += 1
                yield json.loads(line)

#### Чтение данных в цикле

Чтение всех данных занимает 1-2 минуты, поэтому для начала можно попробовать следующее:

1. Читать 10-50 игр
2. Написать код для работы с этими JSON объектами
3. Убедиться, что все работает
4. Запустить код на всем датасете
5. Сохранить результат в `pickle` файл, чтобы в следующий раз не переделывать все заново

## <left>Feature engineering

Напишем функцию, которая поможет нам легче добавлять новые признаки.

In [166]:
def add_new_features(df_features, matches_file, total_matches):
    """
    Аргуенты
    -------
    df_features: таблица с данными
    matches_file: JSON файл с сырыми данными
    
    Результат
    ---------
    Добавляет новые признаки в таблицу
    """
    
    for match in read_matches(matches_file, total_matches):
        match_id_hash = match['match_id_hash']

        # Посчитаем количество разрушенных вышек обеими командами
        radiant_tower_kills = 0
        dire_tower_kills = 0
        for objective in match["objectives"]:
            if objective["type"] == "CHAT_MESSAGE_TOWER_KILL":
                if objective["team"] == 2:
                    radiant_tower_kills += 1
                if objective["team"] == 3:
                    dire_tower_kills += 1

        df_features.loc[match_id_hash, "radiant_tower_kills"] = radiant_tower_kills
        df_features.loc[match_id_hash, "dire_tower_kills"] = dire_tower_kills
        df_features.loc[match_id_hash, "diff_tower_kills"] = radiant_tower_kills - dire_tower_kills

        for idx, player in enumerate(match["players"]):
            deaths = player["deaths"]
            if deaths == 0:
                deaths = 1
            df_features.loc[match_id_hash, f"KDA_{idx}"] = (player["kills"] + player["assists"]) / deaths
            #df_features.loc[match_id_hash, f"ability_uses_{idx}"] = sum(player["ability_uses"].values())

        
        
        # ... (/¯◡ ‿ ◡)/¯☆*:・ﾟ добавляем новые признаки ...

In [167]:
# Скопируем таблицу с признаками
df_train_features_extended = df_train_features.copy()

# Добавим новые
add_new_features(df_train_features_extended, 
                 os.path.join(PATH_TO_DATA, 
                              "train_raw_data.jsonl"),
                 total_matches=31698)

  0%|          | 0/31698 [00:00<?, ?it/s]

In [168]:
# Test
df_test_features_extended = df_test_features.copy()

add_new_features(df_test_features_extended, 
                 os.path.join(PATH_TO_DATA, 
                              "test_raw_data.jsonl"),
                 total_matches=7977)

  0%|          | 0/7977 [00:00<?, ?it/s]

## <left>Наша первая модель

In [169]:
df_train_features_extended.drop(columns=['game_mode', 'lobby_type'], inplace=True)
df_test_features_extended.drop(columns=['game_mode', 'lobby_type'], inplace=True)

In [170]:
id_columns = ['r1_hero_id', 'r2_hero_id', 'r3_hero_id', 'r4_hero_id', 'r5_hero_id', 'd1_hero_id',
             'd2_hero_id', 'd3_hero_id', 'd4_hero_id', 'd5_hero_id']

In [171]:
ct = ColumnTransformer(transformers=[("ohe", OneHotEncoder(drop="first"), id_columns)], remainder='passthrough')

In [172]:
X_ohe = ct.fit_transform(df_train_features_extended)

In [173]:
X_test_ohe = ct.transform(df_test_features_extended)

In [174]:
#X = df_train_features.values
y = df_train_targets["radiant_win"].values.astype("int8")

## <left>Кросс-валидация - подбираем фичи

Во многих случаях кросс-валидация оказывается лучше простого разбиения на test и train. Воспользуемся `ShuffleSplit` чтобы создать 5 70%/30% наборов данных.

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=SEED)

In [ ]:
%%time
rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
cv_scores_rf = cross_val_score(rf_model, X_ohe, y, cv=cv, scoring="roc_auc")

CPU times: user 8.92 s, sys: 540 ms, total: 9.46 s
Wall time: 1min 27s


In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации baseline: {cv_scores_rf.mean()}")

Среднее значение ROC-AUC на кросс-валидации baseline: 0.7720210676055513


In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации (без game_mode, lobby_type): {cv_scores_rf.mean()}")

Среднее значение ROC-AUC на кросс-валидации (без game_mode, lobby_type): 0.7722427312022381


In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации (без game_mode, lobby_type, id): {cv_scores_rf.mean()}")

Среднее значение ROC-AUC на кросс-валидации (без game_mode, lobby_type, id): 0.7723945991018251


In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации (без game_mode, lobby_type + id - one-hot-encoded): {cv_scores_rf.mean()}")

Среднее значение ROC-AUC на кросс-валидации (без game_mode, lobby_type + id - one-hot-encoded): 0.775843473810893


In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации (без game_mode, lobby_type + id - one-hot-encoded + tower features): {cv_scores_rf.mean()}")

Среднее значение ROC-AUC на кросс-валидации (без game_mode, lobby_type + id - one-hot-encoded + tower features): 0.7801170456423246


In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации (без game_mode, lobby_type + id - one-hot-encoded + tower features + KDA): {cv_scores_rf.mean()}")

Среднее значение ROC-AUC на кросс-валидации (без game_mode, lobby_type + id - one-hot-encoded + tower features + KDA): 0.7851820680807531


Ваааааааауууууууууууу

In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации (без game_mode, lobby_type + id - one-hot-encoded + tower features + \n KDA + sum of ability_uses): {cv_scores_rf.mean()}")

Среднее значение ROC-AUC на кросс-валидации (без game_mode, lobby_type + id - one-hot-encoded + tower features + 
 KDA + sum of ability_uses): 0.7848368945271874


Не помогло

## Optuna

In [ ]:
def objective(trial):
    """
    Objective function to be optimized.
    """
    param = {
        "n_estimators": 200,
        #criterion": trial.suggest_categorical("criterion", ['gini', 'entropy', 'log_loss']),
        "max_depth": trial.suggest_int("max_depth", 3, 60),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 40),
    }
    
    cv = KFold(n_splits=3)
    rf_model = RandomForestClassifier(**param, random_state=SEED, n_jobs=-1)

    cv_scores_rf = cross_val_score(rf_model, X_ohe, y, cv=cv, scoring="roc_auc")

    return cv_scores_rf.mean()

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

#clear_output()

In [ ]:
study.best_params, study.best_value

({'max_depth': 27, 'min_samples_split': 27}, 0.7947916762191238)

In [ ]:
params = {'max_depth': 27, 'min_samples_split': 27}

## Увеличим количество деревьев

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=SEED)

In [ ]:
%%time
rf_model = RandomForestClassifier(**params, n_estimators=1500, n_jobs=-1, random_state=SEED)
cv_scores_rf = cross_val_score(rf_model, X_ohe, y, cv=cv, scoring="roc_auc")

CPU times: user 1min 9s, sys: 3.87 s, total: 1min 13s
Wall time: 32min 33s


In [ ]:
cv_scores_rf

array([0.79625372, 0.79911942, 0.78910944, 0.79611321, 0.79550278])

In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации, n estimators = 200: {cv_scores_rf.mean()}")

Среднее значение ROC-AUC на кросс-валидации, n estimators = 200: 0.7927691993663354


In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации, n estimators = 500: {cv_scores_rf.mean()}")

Среднее значение ROC-AUC на кросс-валидации, n estimators = 500: 0.7942713814136482


In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации, n estimators = 1000: {cv_scores_rf.mean()}")

Среднее значение ROC-AUC на кросс-валидации, n estimators = 1000: 0.7949606181858409


In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации, n estimators = 1500: {cv_scores_rf.mean()}")

Среднее значение ROC-AUC на кросс-валидации, n estimators = 1500: 0.7952197124176128


In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации, n estimators = 2000: {cv_scores_rf.mean()}")

Среднее значение ROC-AUC на кросс-валидации, n estimators = 2000: 0.7953231552699084


Кажентся, 1500 - оптимально

## Обучим случайный лес

In [ ]:
%%time
rf_model = RandomForestClassifier(**params, n_estimators=1500, n_jobs=-1, random_state=SEED)
rf_model.fit(X_ohe, y)

CPU times: user 17min 18s, sys: 856 ms, total: 17min 18s
Wall time: 9min 57s


RandomForestClassifier(max_depth=27, min_samples_split=27, n_estimators=1500,
                       n_jobs=-1, random_state=10801)

## <left>Посылка

In [ ]:
y_test_pred = rf_model.predict_proba(X_test_ohe)[:, 1]

df_submission = pd.DataFrame({"radiant_win_prob": y_test_pred}, 
                                 index=df_test_features.index)

In [ ]:
submission_filename = "submission_{}.csv".format(
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
df_submission.to_csv(submission_filename)
print("Файл посылки сохранен, как: {}".format(submission_filename))

Файл посылки сохранен, как: submission_2023-04-09_13-34-04.csv


## Catboost

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df_train_features_extended, 
                                                      y, 
                                                      test_size=0.3, 
                                                      random_state=SEED)

In [ ]:
id_columns = ['r1_hero_id', 'r2_hero_id', 'r3_hero_id', 'r4_hero_id', 'r5_hero_id', 'd1_hero_id',
             'd2_hero_id', 'd3_hero_id', 'd4_hero_id', 'd5_hero_id']

In [ ]:
ct_classifier = CatBoostClassifier(cat_features=id_columns)

In [ ]:
ct_classifier.fit(X_train, y_train)

In [ ]:
y_pred = ct_classifier.predict_proba(X_valid)[:, 1]

In [ ]:
valid_score = roc_auc_score(y_valid, y_pred)
print("ROC-AUC score на отложенной части:", valid_score)

ROC-AUC score на отложенной части: 0.8044098232288559


In [ ]:
y_test_pred = ct_classifier.predict_proba(df_test_features_extended)[:, 1]

df_submission = pd.DataFrame({"radiant_win_prob": y_test_pred}, 
                                 index=df_test_features.index)

In [ ]:
submission_filename = "submission_{}.csv".format(
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
df_submission.to_csv(submission_filename)
print("Файл посылки сохранен, как: {}".format(submission_filename))

Файл посылки сохранен, как: submission_2023-04-09_14-04-48.csv


## LightGBM

In [175]:
from lightgbm import LGBMClassifier

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=SEED)

In [ ]:
%%time
lgbm_model = LGBMClassifier(n_jobs=-1, random_state=SEED)
cv_scores_rf = cross_val_score(lgbm_model, X_ohe, y, cv=cv, scoring="roc_auc")

CPU times: user 41.1 s, sys: 188 ms, total: 41.3 s
Wall time: 25.1 s


In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации baseline: {cv_scores_rf.mean()}")

Среднее значение ROC-AUC на кросс-валидации baseline: 0.7998659813374207


In [ ]:
def objective(trial):
    """
    Objective function to be optimized.
    """
    param = {
        "num_leaves": trial.suggest_int("num_leaves", 10, 60),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.05, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000)
        #"min_split_gain": trial.suggest_float("min_split_gain", 0, 1),
        #"min_child_weight": trial.suggest_float("min_split_gain", 0, 1),
    }
    cv = KFold(n_splits=5)
    lgbm = LGBMClassifier(**param)
    score = cross_val_score(lgbm, X_ohe, y, scoring='roc_auc', cv=cv)
    
    return score.mean()

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

clear_output()

In [ ]:
study.best_params, study.best_value

({'num_leaves': 39,
  'max_depth': 8,
  'learning_rate': 0.053159958057109824,
  'n_estimators': 866},
 0.808033477878291)

In [ ]:
params = study.best_params

In [176]:
params = {'num_leaves': 39,
        'max_depth': 8,
        'learning_rate': 0.053159958057109824,
        'n_estimators': 866}

In [177]:
%%time
lgbm = LGBMClassifier(**params)
lgbm.fit(X_ohe, y)

CPU times: user 56.4 s, sys: 236 ms, total: 56.6 s
Wall time: 33.6 s


LGBMClassifier(learning_rate=0.053159958057109824, max_depth=8,
               n_estimators=866, num_leaves=39)

In [178]:
y_test_pred = lgbm.predict_proba(X_test_ohe)[:, 1]

df_submission = pd.DataFrame({"radiant_win_prob": y_test_pred}, 
                                 index=df_test_features.index)

In [179]:
submission_filename = "submission_{}.csv".format(
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
df_submission.to_csv(submission_filename)
print("Файл посылки сохранен, как: {}".format(submission_filename))

Файл посылки сохранен, как: submission_2023-04-09_17-32-26.csv
